In [2]:
import numpy as np
import rqcopt as oc
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
import jax
from jax import config
config.update("jax_enable_x64", True)
import jax.numpy as jnp
import scipy

In [44]:
# setup for problem
# get the setup for the problem
from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, super2choi
from opentn.optimization import compose_superops_list, frobenius_norm
d, N, gamma = 2, 4, 1
tau = 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=gamma)
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

# 2 site superoperator ansatz
from opentn.transformations import lindbladian2super, create_supertensored_from_local
superop_nn = lindbladian2super(Li=[Lnn])
exp_nn_odd = exp_operator_dt(superop_nn, tau=tau/2, library='jax')
exp_nn_even = exp_operator_dt(superop_nn, tau=tau, library='jax')

print(frobenius_norm(compose_superops_list([exp_Lvec_odd, exp_Lvec_even, exp_Lvec_odd]), exp_Lvec))

0.0959176702323516


In [29]:
# TODO: missing to create the model that takes 3 ws and turns them into 3 full Ys.
# TODO: model Zs does something similar but only for 1 layer.
# the one for the middle layer should somehow look different in any case, because it needs to be shifted with the periodic boundary condition.
# since the exp is anyways left without the BC, for now I think I can just ommit it (as usual
# TODO: the cost function should have the input as isometries.

In [30]:
from opentn.states.qubits import X
def permute_operation(U: np.ndarray, perm):
    """
    Find the representation of a matrix after permuting lattice sites.
    """
    nsites = len(perm)
    assert U.shape == (2**nsites, 2**nsites)
    perm = list(perm)
    U = np.reshape(U, (2*nsites) * (2,))
    U = np.transpose(U, perm + [nsites + p for p in perm])
    U = np.reshape(U, (2**nsites, 2**nsites))
    return U
permute_operation(np.kron(np.eye(2),X), [1,0])


array([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]])

In [31]:
# how would we vectorize the identity?
from opentn.transformations import vectorize_dissipative, vectorize_hamiltonian, dissipative2liouvillian_full
Inn = np.eye(d**2)
print(np.allclose(vectorize_dissipative(Inn), np.zeros((d**4,d**4))), np.allclose(vectorize_hamiltonian(Inn), np.zeros((d**4,d**4))))
# as wrote out on paper, the identity "vectorized" is just zero
print(dissipative2liouvillian_full(L=Lnn, i=1, N=N, num_sites=2).shape)

True True
(256, 256)


In [32]:
from opentn.transformations import convert_supertensored2liouvillianfull
I_rest = np.eye(d**4,d**4)
# corresponds to site order: (N-1, 0), (1, 2)
even_full_test = np.kron(I_rest, exp_nn_even) 
swaped_test = convert_supertensored2liouvillianfull(even_full_test, N, d, pbc=True)
np.allclose(swaped_test, exp_Lvec_even)

True

In [33]:
# now another question I have is even the components I am using would make isometries. CHECK
from opentn.transformations import factorize_psd_truncated, choi2ortho
from opentn.stiefel import is_isometry_2
print(is_isometry_2(choi2ortho(factorize_psd_truncated(super2choi(exp_nn_even.real), chi_max=2))))
print(is_isometry_2(choi2ortho(factorize_psd_truncated(super2choi(I_rest), chi_max=d**4))))

True
True


In [34]:
test_odd = create_supertensored_from_local(exp_nn_odd, N)
test_odd = convert_supertensored2liouvillianfull(test_odd, N, d)
np.allclose(test_odd@swaped_test@test_odd, exp_Lvec_odd@exp_Lvec_even@exp_Lvec_odd) # AMAZING

True

# Next steps:
(18 OCT 2023)

1. I want the above functions to be a comprehensible framework. What does this mean?
    a. Their should be one "factory" that takes superop_local -> super_full_liouvillian that takes as argument the parity and shifts (PBC) for the even layer.
    b. I need to try this out fully when also changing the exp(L_full) to include the PBC for the term [N-1,0].
    c. Include a special case for the non PBC such that this old comparison still works
    c. Add even layer to unit tests 
2. Try optimization with the PBC case for St(N)
3. EXTRA: finish creation of identity maps!

In [35]:
# I need to change now create_2local_liouvillians to include pbc condition
from opentn.transformations import permute_cyclic
permute_cyclic(list(range(6)), direction='right')

# NOTE: a unittest I can write is constructing the same operator once with left and once with right!

[5, 0, 1, 2, 3, 4]

In [42]:
# trying out again things but with pbc=True!
from opentn.transformations import create_trotter_layers
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=gamma, pbc=True)
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = create_trotter_layers(liouvillians=[Lvec, Lvec_odd, Lvec_even], tau=tau)

superop_nn = lindbladian2super(Li=[Lnn])
exp_nn_odd = exp_operator_dt(superop_nn, tau=tau/2, library='jax')
exp_nn_even = exp_operator_dt(superop_nn, tau=tau, library='jax')

test_even_pbc = create_supertensored_from_local(exp_nn_even, N)
test_even_pbc = convert_supertensored2liouvillianfull(test_even_pbc, N, d, pbc=True)

test_odd = create_supertensored_from_local(exp_nn_odd, N)
test_odd = convert_supertensored2liouvillianfull(test_odd, N, d)

In [43]:
np.allclose(test_odd@test_even_pbc@test_odd, exp_Lvec_odd@exp_Lvec_even@exp_Lvec_odd) # AMAZING

True